In [2]:
import pandas as pd

In [4]:
df = pd.read_csv("../input/train_folds.csv")
df.head(3)

,Unnamed: 0,id,target,file_path,fold
0,0,00000e74ad,1,input/train/0/0/0/00000e74ad.npy,2
1,1,00001f4945,0,input/train/0/0/0/00001f4945.npy,1
2,2,0000661522,0,input/train/0/0/0/0000661522.npy,3


In [5]:
import torch

In [8]:
states_1d = [torch.load(f"../final_models/b7_1d/tf_efficientnet_b7_ns_fold{fold}_best_score.pth") for fold in range(5)]

In [14]:
states_3d = [torch.load(f"../final_models/b7_3d/tf_efficientnet_b7_ns_fold{fold}_best_score.pth") for fold in range(5)]

In [10]:
states_1d[0]['preds'].flatten()

array([0.3762, 0.2048, 0.745 , ..., 0.1508, 0.2474, 0.3853], dtype=float16)

In [25]:
oof_df = pd.DataFrame()
for fold in range(5):
    preds_1d = states_1d[fold]['preds'].flatten()
    preds_3d = states_3d[fold]['preds'].flatten()
    _df = df[df['fold'] == fold].reset_index(drop=True)
    _df['preds_1d'] = preds_1d
    _df['preds_3d'] = preds_3d
    oof_df = pd.concat([oof_df, _df])

In [26]:
oof_df

,Unnamed: 0,id,target,file_path,fold,preds_1d,preds_3d
0,9,00010beb4a,1,input/train/0/0/0/00010beb4a.npy,0,0.376221,0.378174
1,12,00014b7a9d,1,input/train/0/0/0/00014b7a9d.npy,0,0.204834,0.196777
2,17,0001c76f56,1,input/train/0/0/0/0001c76f56.npy,0,0.745117,0.497070
3,31,0002b883f5,0,input/train/0/0/0/0002b883f5.npy,0,0.191895,0.239746
4,37,00034d8429,0,input/train/0/0/0/00034d8429.npy,0,0.217163,0.168335
...,...,...,...,...,...,...,...
111995,559973,fffde18882,0,input/train/f/f/f/fffde18882.npy,4,0.240601,0.279053
111996,559974,fffdf87619,1,input/train/f/f/f/fffdf87619.npy,4,1.000000,1.000000
111997,559979,fffe81f2ec,0,input/train/f/f/f/fffe81f2ec.npy,4,0.166382,0.161011
111998,559988,ffff1a0fee,1,input/train/f/f/f/ffff1a0fee.npy,4,0.824707,0.999023


In [27]:
from sklearn.metrics import roc_auc_score

In [28]:
A = oof_df.preds_1d.values
B = oof_df.preds_3d.values
targets = oof_df.target.values
for i in range(1, 100):
    a = i*0.01
    b = 1 - a

    y_hat = a*A + b*B
    score = roc_auc_score(targets, y_hat)

    print(i*0.01, score)

0.01 0.8743197996130997
0.02 0.8743944000195479
0.03 0.8744637078300022
0.04 0.8745329158445316
0.05 0.8745966929212219
0.06 0.8746594628294817
0.07 0.8747190161622853
0.08 0.8747758215995092
0.09 0.8748294915645983
0.1 0.8748804066634438
0.11 0.8749299898616959
0.12 0.8749766616439496
0.13 0.8750205751479696
0.14 0.8750619866811701
0.15 0.8751008592027323
0.16 0.8751370743899447
0.17 0.8751711139902865
0.18 0.8752028607333421
0.19 0.8752315363728894
0.2 0.8752585383452404
0.21 0.8752822485699365
0.22 0.8753043409310111
0.23 0.8753241031643891
0.24 0.8753414230761859
0.25 0.8753561079750629
0.26 0.875367818441356
0.27 0.8753788865670478
0.28 0.8753861421032154
0.29 0.8753921886214506
0.3 0.87539513243541
0.31 0.8753962605222152
0.32 0.8753948619506993
0.33 0.8753915362936038
0.34 0.8753849743225304
0.35000000000000003 0.8753778507889218
0.36 0.8753666427971498
0.37 0.8753545614826195
0.38 0.875339850824812
0.39 0.8753219580685918
0.4 0.875303004916897
0.41000000000000003 0.875281140961